# Adding Software Specification to WML

This notebook's goal is to showcase how you can extend WML's existing Software Specification and how it can be useful for your Decision Optimization experiment.

In this example, a new Python package is added and used in the optimization model.

## Importing and instantiating WML

The first step is to add WML to your notebook. It is not imported by default, so you will have to add it manually. To do so, you can use the following code:

In [ ]:
from ibm_watson_machine_learning import APIClient

In [ ]:
# Instantiating WML's client using our credentials. Don't forget to change this part with your own credentials!
# You might want to modify the instance_id, depending on where you are using this example
cluster = "<your_cluster>"
username = "<username>"
password = "<password>"

wml_credentials = {
    "username": username,
    "password": password,
    "instance_id" : "wml_local",
    "url": cluster,
    "version": "4.8"
}

client = APIClient(wml_credentials)

In order to work, WML must also be given what's called a `space`, that is, the place where to deploy your model to. You might already have created a few spaces. You can check if that is the case using the following code. It will display all the spaces that you have created that currently exist on the platform.

In [ ]:
def guid_from_space_name(client, space_name):
     space = client.spaces.get_details()
     return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])


client.spaces.list()

You can then find one space that you wish to use, and execute the following code to tell WML to use it.

In [ ]:
# Select a space from the list displayed above
space_id = guid_from_space_name(client,"<space_name>")
client.set.default_space(space_id)

## Creating a simple package extension

For the purpose of this demonstration, you will create a very simple package extension that will install the pip package called `hello_demo`. Of course, feel free to replace that by whatever you might need.

The first step is to write a small `yaml` file, here named `main.yml`, for this package extension, like so:

In [ ]:
%%writefile main.yml

name: do_example
channels:
  - defaults
dependencies:
  - pip:
    - hello_demo

Once done, you can store it in the package extensions using `client.package_extensions.store(meta_props=meta_prop_pkg_ext, file_path="/home/wsuser/work/main.yml")`

You can also store the uid of the extension for later usage, using `client.package_extensions.get_uid()`

In [ ]:
# These first few lines, makes the name of the package unique using the current time
import time
current_time = time.asctime()

meta_prop_pkg_ext = {
     client.package_extensions.ConfigurationMetaNames.NAME: "conda_ext_" + current_time,
     client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Pkg extension for conda",
     client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml",
}

# Storing the package and saving it's uid
pkg_ext_id = client.package_extensions.get_uid(client.package_extensions.store(meta_props=meta_prop_pkg_ext,
                                         file_path="/home/wsuser/work/main.yml"))



### Extend DO V22.1 existing software specification with the package extension created previously

You now want to create a DO Model that is going to use the pip package from the package extension and use it. 

First of all, create a new model, and print the pip package version number. The model `main.py` will be:

```
import hello_demo

print("hello_demo version: " + hello_demo.__version__)
```

In [ ]:
%mkdir -p model

In [ ]:
%%writefile model/main.py

import hello_demo

print("hello_demo version: " + hello_demo.__version__)

You now need to compress the model directory you created with tar, so that it can be deployed in WML. That is what the next cell does.

In [ ]:
# Creating a tar from the model you just created
import tarfile
def reset(tarinfo):
    tarinfo.uid = tarinfo.gid = 0
    tarinfo.uname = tarinfo.gname = "root"
    return tarinfo
tar = tarfile.open("model.tar.gz", "w:gz")
tar.add("model/main.py", arcname="main.py", filter=reset)
tar.close()

Done! This model is ready for use! 

Since the model is using a custom pip package that is not available by default in DO V22.1, you need to extend its' software specifications.

To do so, use the following code. It will create an extension to the current specifications of DO V22.1 and add the package you previously created, making the `hello_demo` package available to your model.

In [ ]:
# Look for the do_22.1 software specification
base_sw_id = client.software_specifications.get_uid_by_name("do_22.1")

# Create a new software specification using the default do_22.1 one as the base for it
meta_prop_sw_spec = {
    client.software_specifications.ConfigurationMetaNames.NAME: "do_22.1_ext_"+current_time,
    client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for DO example",
    client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": base_sw_id}
}
sw_spec_id = client.software_specifications.get_uid(client.software_specifications.store(meta_props=meta_prop_sw_spec)) # Creating the new software specification
client.software_specifications.add_package_extension(sw_spec_id, pkg_ext_id) # Adding the previously created package extension to it

Time to test everything! You can now store your model in WML, deploy it and then run it.

When storing the model, that is where you must specify the new software specification to use, the one you just created. As you can see, you add the ID within the metadata used to store the model, in `client.repository.ModelMetaNames.SOFTWARE_SPEC_UID`

In [ ]:
# Storing it with custom metadata, feel free to change this part...
mnist_metadata = {
    client.repository.ModelMetaNames.NAME: "Extend WML software spec",
    client.repository.ModelMetaNames.DESCRIPTION: "Model for extend WML software specification",
    client.repository.ModelMetaNames.TYPE: "do-docplex_22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_id    
}

model_details = client.repository.store_model(model='/home/wsuser/work/model.tar.gz', meta_props=mnist_metadata)

model_uid = client.repository.get_model_id(model_details)

print(model_uid)

In [ ]:
# Deploying the model
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "Extend WML software spec deployment",
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "Extend WML software specification deployment",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"name" : "S", "num_nodes":1 }
}

deployment_details = client.deployments.create(model_uid, meta_props=meta_props)
deployment_uid = client.deployments.get_uid(deployment_details)

print(deployment_uid)

The next few cells create the WML job for this model and wait for it to be solved. Once solved, logs are displayed, where you should see the `hello_demo` version number.

In [ ]:
solve_payload = {
    client.deployments.DecisionOptimizationMetaNames.SOLVE_PARAMETERS: {
        "oaas.logTailEnabled":"true"
    }
}

job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)

print(job_uid)

In [ ]:
from time import sleep

while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    job_details=client.deployments.get_job_details(job_uid)

print( job_details['entity']['decision_optimization']['solve_state']['latest_engine_activity'])

As you can see, it worked, there are no errors. Moreover, we can see that the version of the package has been successfully printed in the logs, confirming that is indeed installed. Without the added software specifications, the output of the run would have been: "ModuleNotFoundError : No module named 'hello_demo'"

But, with the added software specification extension, this pip package is indeed installed, and we don't have this error message anymore.

## Conclusion

This example has shown you how to extend the software specification of DO V22.1 within WML and how it can be useful. That way, you can add even more custom code and use it within your model in a very simple manner, extending DO's capacity to fit your needs.


<hr>
Copyright &copy; 2017-2023 IBM. This notebook and its source code are released under the terms of the MIT License.